In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader


In [3]:
df = pd.read_csv('dataset.csv.gz')


In [39]:
df_train, df_test = df[df['Train'] == 1], df[df['Train'] == 0]

device_names = set([n for n in df['DeviceName'] if isinstance(n, str)])
device_name_encoding = dict(zip(device_names, range(len(device_names))))
print('Device Name Encoding:', device_name_encoding)


def calc_intersection(lx, ly, lz, l0x, l0y, l0z):
    # https://zh.m.wikipedia.org/zh-hans/%E7%BA%BF%E9%9D%A2%E4%BA%A4%E7%82%B9
    # d = (p0 - l0) \cdot n / (l \cdot n)
    # where p0 = (0, 0, 0)
    #   and n = (0, 0, 1)
    d = -l0z / lz
    # p = dl + l0
    return d * lx + l0x, d * ly + l0y


def process(df):
    df = df[df[df.columns].notnull().all(1)]
    df = df.drop(columns=['Train'])
    for device_name in device_names:
        df.loc[df['DeviceName'] == device_name, device_name] = True
        df.loc[df['DeviceName'] != device_name, device_name] = False

    for orientation in [1, 2, 3, 4]:
        df.loc[df['Orientation'] == orientation, f'Orientation_{orientation}'] = True
        df.loc[df['Orientation'] != orientation, f'Orientation_{orientation}'] = False

    df['gaze_point_0_x'], df['gaze_point_0_y'] = calc_intersection(df['gaze_0_x'], df['gaze_0_y'], df['gaze_0_z'], df['eye_lmk_0_X'], df['eye_lmk_0_Y'], df['eye_lmk_0_Z'])
    df['gaze_point_1_x'], df['gaze_point_1_y'] = calc_intersection(df['gaze_1_x'], df['gaze_1_y'], df['gaze_1_z'], df['eye_lmk_1_X'], df['eye_lmk_1_Y'], df['eye_lmk_1_Z'])

    df = df.drop(columns=['DeviceName', 'Orientation'])

    # shift target labels to the end
    cols = [c for c in df.columns if not c in ['XCam', 'YCam']]
    cols += ['XCam', 'YCam']
    return df[cols]

df_train, df_test = process(df_train), process(df_test)


Device Name Encoding: {'iPhone 6': 0, 'iPhone 6s Plus': 1, 'iPad 3': 2, 'iPad 2': 3, 'iPad Air': 4, 'iPad Air 2': 5, 'iPhone 4S': 6, 'iPad 4': 7, 'iPad Mini': 8, 'iPhone 5S': 9, 'iPhone 5C': 10, 'iPad Pro': 11, 'iPhone 6 Plus': 12, 'iPhone 6s': 13, 'iPhone 5': 14}


In [40]:
pd.set_option('display.max_columns', None)

df_train.head()

,eye_lmk_0_X,eye_lmk_0_Y,eye_lmk_0_Z,eye_lmk_1_X,eye_lmk_1_Y,eye_lmk_1_Z,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,H,W,iPhone 6,iPhone 6s Plus,iPad 3,iPad 2,iPad Air,iPad Air 2,iPhone 4S,iPad 4,iPad Mini,iPhone 5S,iPhone 5C,iPad Pro,iPhone 6 Plus,iPhone 6s,iPhone 5,Orientation_1,Orientation_2,Orientation_3,Orientation_4,gaze_point_0_x,gaze_point_0_y,gaze_point_1_x,gaze_point_1_y,XCam,YCam
0,-24.2500,-6.1125,392.2625,29.550,-8.1500,357.5000,0.072223,-0.316301,-0.945906,-0.066284,-0.306683,-0.949501,0.003,-0.318,1024.0,768.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,5.700518,-137.280947,4.593175,-123.620308,-6.673958,-20.086172
1,-23.5000,-39.8750,383.2500,33.350,-42.2250,379.6875,0.015097,-0.178681,-0.983791,-0.131029,-0.046250,-0.990299,-0.059,-0.113,1024.0,768.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,-17.618745,-109.482766,-16.887427,-59.957571,-6.673958,-20.086172
2,-24.9500,-49.4000,371.9000,31.975,-53.6000,382.7375,0.016881,-0.112260,-0.993536,-0.091935,-0.015112,-0.995650,-0.038,-0.064,1024.0,768.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,-18.631111,-91.421119,-3.365704,-59.409199,-6.673958,-20.086172
3,49.3125,-3.4375,344.8000,104.175,-6.7750,346.6000,-0.091046,-0.207954,-0.973892,-0.117662,-0.125064,-0.985147,-0.106,-0.168,1024.0,768.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,17.078268,-77.062234,62.778488,-50.775725,-3.752047,-13.512109
4,60.4000,-18.9000,395.0500,103.475,-18.9625,336.4000,-0.096623,-0.090880,-0.991163,-0.157676,-0.082833,-0.984011,-0.128,-0.088,1024.0,768.0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,21.888760,-55.122240,49.570921,-47.280294,-3.752047,-13.512109


In [48]:
train_tensor, test_tensor = torch.tensor(df_train.values.astype(float)).float(), torch.tensor(df_test.values.astype(float)).float()

train_dataset = TensorDataset(train_tensor[:, :-2], train_tensor[:, -2:])
test_dataset = TensorDataset(test_tensor[:, :-2], test_tensor[:, -2:])


In [52]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(39, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
        )

    def forward(self, x):
        return self.mlp(x)


batch_size = 512

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)

device = 'cpu'

model = Model().to(device)

print(model(train_dataset[0][0].to(device)))

def loss_fn(y1, y2):
    return (y1 - y2).pow(2).sum(dim=-1).sqrt().mean()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def train(dataloader, model, loss_fn, optimizer, debug=False):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 2000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        if debug and batch % 2000 == 0 and batch > 0:
            print('pred', pred)
            print('y', y)
            print('Loss', loss)
            break

def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Test loss: {test_loss:>8f} \n")


train(train_dataloader, model, loss_fn, optimizer, True)

epochs = 50
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    torch.save(model.state_dict(), f'checkpoints/ckpt-{i+1}.pt')
print("Done!")

tensor([ 0.3235, -4.0627], grad_fn=<AddBackward0>)
loss: 6.902768  [    0/1932042]
loss: 4.012379  [1024000/1932042]
pred tensor([[  5.2267,   0.5229],
        [-11.1278,  -0.4404],
        [  6.0903,   0.6708],
        ...,
        [ -0.5411,   0.2053],
        [  0.3524,  -5.3242],
        [  5.5823,   0.4189]], grad_fn=<AddmmBackward0>)
y tensor([[  9.4675,  -1.9388],
        [-16.5418,  -3.7372],
        [  1.4270,  -1.2370],
        ...,
        [ -4.8972, -14.1373],
        [  0.9694,  -3.6430],
        [ 11.7054,  -0.1029]])
Loss 4.012379169464111
Epoch 1
-------------------------------
loss: 4.035036  [    0/1932042]
loss: 3.633030  [1024000/1932042]
Test loss: 3.631090 

Epoch 2
-------------------------------
loss: 3.316515  [    0/1932042]
loss: 3.545309  [1024000/1932042]
Test loss: 3.934872 

Epoch 3
-------------------------------
loss: 3.931549  [    0/1932042]
loss: 3.348649  [1024000/1932042]
Test loss: 3.614380 

Epoch 4
-------------------------------
loss: 3.545483 